In [63]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import optuna
from sklearn.ensemble import StackingClassifier

from mlxtend.plotting import plot_learning_curves
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling  import RandomOverSampler
from sklearn.feature_selection import RFE
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score ,confusion_matrix
from sklearn.decomposition import PCA
from vecstack import stacking

In [64]:
df = pd.read_csv('wtrain_data.csv')
test=pd.read_csv('wtest_data.csv')
labels_df = pd.read_csv("solution_train.csv",sep=',')

print(f' Shape of  Dataset: {df.shape}')

 Shape of  Dataset: (566, 52)


In [65]:
X = df.drop(['date','label'],axis=1)
y = df['label']

print(f' X_shape: {X.shape} \n y_shape: {y.shape}')

 X_shape: (566, 50) 
 y_shape: (566,)


In [66]:
rf_clf = RandomForestClassifier(n_estimators=32)
rfe_selector = RFE(estimator=rf_clf, n_features_to_select=20, step=5, verbose=5)
rfe_selector.fit(X, y)

Fitting estimator with 50 features.
Fitting estimator with 45 features.
Fitting estimator with 40 features.
Fitting estimator with 35 features.
Fitting estimator with 30 features.
Fitting estimator with 25 features.


RFE(estimator=RandomForestClassifier(n_estimators=32), n_features_to_select=20,
    step=5, verbose=5)

In [67]:
rfe_support = rfe_selector.get_support()

rfe_feature = X.loc[:,rfe_support].columns.tolist()
print(rfe_feature)
print('*******************************************************************************************')
print(str(len(rfe_feature)), 'selected features')

['avg.temp_B', 'avg.temp_D', 'avg.wind.speed_A', 'avg.wind.speed_D', 'avg.wind.speed_E', 'max.inst.wind.speed_A', 'max.inst.wind.speed_B', 'max.inst.wind.speed_C', 'max.inst.wind.speed_D', 'max.inst.wind.speed_E', 'max.wind.speed_A', 'max.wind.speed_B', 'max.wind.speed_D', 'max.wind.speed_E', 'min.atmos.pressure_A', 'min.temp_A', 'min.temp_B', 'min.temp_E', 'precipitation_A', 'precipitation_D']
*******************************************************************************************
20 selected features


In [68]:
X  = df[rfe_feature]
y = df['label']

print(f' X_shape: {X.shape} \n y_shape: {y.shape}')

 X_shape: (566, 20) 
 y_shape: (566,)


In [69]:
rs = RandomOverSampler()
X_resample, y_resample = rs.fit_resample(X,y)

print(f' Shape of X after Oversampling: {X_resample.shape} \n Shape of y after Oversampling: {y_resample.shape}')

 Shape of X after Oversampling: (1062, 20) 
 Shape of y after Oversampling: (1062,)


In [70]:
from sklearn.model_selection import train_test_split
X_train_res, X_test_res, y_train_res, y_test_res = train_test_split(X_rescale,y_resample,test_size=0.20,shuffle=True,random_state=21)
print(f' Shape of X_train: {X_train_res.shape} & Shape of X_test: {X_test_res.shape}')
print(f' Shape of y_train: {y_train_res.shape} & Shape of y_test: {y_test_res.shape}')

 Shape of X_train: (849, 20) & Shape of X_test: (213, 20)
 Shape of y_train: (849,) & Shape of y_test: (213,)


In [71]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_res = scaler.fit_transform(X_train_res)
X_test_res = scaler.transform(X_test_res)

In [77]:
from xgboost import XGBClassifier

xgb = XGBClassifier(booster = 'gbtree', learning_rate = 0.1, max_depth = 5, n_estimators = 180)
xgb.fit(X_train_res, y_train_res);

In [78]:
XGBClassifierScore = xgb.score(X_test_res,y_test_res)
print("Accuracy obtained by XGB Classifier model:", XGBClassifierScore*100)

Accuracy obtained by XGB Classifier model: 89.67136150234741


In [79]:
y_pred_xgb = xgb.predict(X_test_res)
print(metrics.classification_report(y_test_res, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98        75
           1       0.81      0.93      0.86        67
           2       0.93      0.76      0.84        71

    accuracy                           0.90       213
   macro avg       0.90      0.90      0.89       213
weighted avg       0.90      0.90      0.90       213

